In [5]:
import urllib.request

import tiktoken

url = ("https://raw.githubusercontent.com/rasbt/"
       "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
       "the-verdict.txt")
file_path = "the-verdict.txt"
urllib.request.urlretrieve(url, file_path)

('the-verdict.txt', <http.client.HTTPMessage at 0x111af5b90>)

In [6]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
print("Total number of characters:", len(raw_text))
print(raw_text[:99])

Total number of characters: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [7]:
import re
text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)
print(result)

# We refrain from making all text lower-case because caps help LLMs distinguish between proper nouns and common nouns, understand sentence structure, and learn to generate text with proper capitalization.

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


In [8]:
result = re.split(r'([,.]|\s)', text)
result = [item for item in result if item.strip()]
print(result)

#whether or not we encode whitespaces as separate characters or just remove them depends on our application and its requirements.
#keeping whitespaces can be useful if we train models that are sensitive to the exact structure of the text (like python code). Removing them reduces memory and computing requirements.

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


In [9]:
text = "Hello, world. Is this-- a test?"
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [10]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(len(preprocessed))
print(preprocessed[:30])

# vocabs define how we map each unique word and special character to a unique integer. each unique token is mapped to a unique token ID.
# each unique token is added to the vocab in alphabetical order. duplicate tokens are removed.

4690
['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [11]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print(vocab_size)

vocab = {token:integer for integer, token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

#starting with a new sample text, we tokenize this new text and use the existing vocabulary to convert the text tokens into token IDs.
# the vocab is built from the entire training set and can be applied to the training set itself and any new text samples.

1130
('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


**A complete tokenizer class** that:

- Splits text into tokens.
- Carries out the string-to-integer mapping to produce token IDs via the vocabulary.
- Implements a `decode` method that carries out the reverse integer-to-string mapping to convert the token IDs back into text.

In [12]:
class SimpleTokenizerV1:
    def __init__(self, vocab): # the function works via an existing vocab, which we can use to encode and decode text.
        self.str_to_int = vocab
        self.int_to_str = {i:s for s, i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text) # preprocesses input text into token IDs
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids]) # converts tokens back into text
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text) #removes spaces before the specified punctuation
        return text

tokenizer = SimpleTokenizerV1(vocab)
text = """It's the last he painted, you know, Mrs.Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)
print(tokenizer.decode(ids))

[56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 67, 7, 38, 851, 1108, 754, 793, 7]
It' s the last he painted, you know, Mrs. Gisburn said with pardonable pride.


In [13]:
text = "Hello, do you like tea?"
print(tokenizer.encode(text)) #word "Hello" not included in "The Verdict" short story. Need to consider larger, diverse training sets to extend the vocabulary when working with LLMs.
# we can just modify the tokenizer to use an <|unk|> token if it encounters a word that is not part of the vocabulary.
# we can also add a <|endoftext|> token that we can use to separate two unrelated text sources. this helps the lLM understand that although these text sources are concatenated for training, they are, in fact, unrelated.

KeyError: 'Hello'

In [14]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token:integer for integer, token in enumerate(all_tokens)}
print(len(vocab.items()))

1132


In [15]:
class SimpleTokenizerV2:
    def __init__(self, vocab): # the function works via an existing vocab, which we can use to encode and decode text.
        self.str_to_int = vocab
        self.int_to_str = {i:s for s, i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text) # preprocesses input text into token IDs
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [item if item in self.str_to_int else "<|unk|>" for item in preprocessed]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids]) # converts tokens back into text
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text) #removes spaces before the specified punctuation
        return text

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1, text2))
print(text)

tokenizer = SimpleTokenizerV2(vocab)
print(tokenizer.encode(text))
print(tokenizer.decode(tokenizer.encode(text))) #quick sanity check

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.
[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]
<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


Depending on the LLM, some researchers also consider additional special tokens such as the following:

- `[BOS]` (beginning of sequence), marks the start of a text. Signifies to the LLM where a piece of content begins.
- `[EOS]` (end of sequence), positioned at the end of a text and is especially useful when concatenating multiple unrelated texts, similar to `<|endoftext|>`.
- `[PAD]` (padding), when training LLMs with batch sizes larger than one, the batch might contain texts of varying length. The shorter texts are extended, padded, up to the length of the longest text in the batch.


In [18]:
import tiktoken

#bpe breaks down words into subword units. Unknown tokens are not longer used, nor needed.
tokenizer = tiktoken.get_encoding("gpt2")
text = ("Helo, do you like tea? <|endoftext|> In the sunlit terraces"
        "of someunknownPlace")
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)
strings = tokenizer.decode(integers)
print(strings)

#the endoftext token is assigned the largest token ID, out of 50257 tokens.
#the BPE encodes and decodes unknown words correctly.

[12621, 78, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271]
Helo, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace


The algorithm underlying byte-pair encoding breaks down words that aren't in its predefined vocabulary into smaller subwords units or even individual characters, enabling it to handle out-of-vocab words.
BPE builds its vocabulary iteratively merging frequent characters into subwords and frequent subwords into words. It starts by adding all individual single characters to its vocab ("a", "b", etc.). Next, it merges character combinations that frequently occur together into subwords, each merge being determined by a frequency cutoff.

**Example:**
- The text sample "Akwir ier" would first be tokenized into individual characters or subwords.
- "Ak", "w", "ir", "w", "", "ier", each with their corresponding token IDs (33901, 86, 343, 86, 220, 959).

The ability to break down words into individual characters ensures that the tokenizer, and the LLM that is trained with it, can process any text, even if it contains words that were not present in its training data.

In [19]:
example = "Awkir ier"
integers_ex = tokenizer.encode(example, allowed_special={"<|endoftext|>"})
string_ex = tokenizer.decode(integers_ex)
print(integers_ex, string_ex)

[23155, 74, 343, 220, 959] Awkir ier


### Data Sampling with a Sliding Window
We now create the input-target pairs required for training the LLM. LLMs are pretrained by predicting the next word in a text, and we mask out all words that are past the target.
**Example:**

- Input: "LLMs", target: "learn"
- Input: "LLMs learn", target: "to"
- Input: "LLMs learn to", target: "predict"
- Input: "LLMs learn to predict", target: "one"
- Input: "LLMs learn to predict one", target: "word"


In [22]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

enc_sample = enc_text[50:]
context_size = 4 #how many tokens are included in the input
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
print(f"x: {x}")
print(f"y:      {y}")

5145
x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [25]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "--->", desired)

for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), "--->", tokenizer.decode([desired]))

[290] ---> 4920
[290, 4920] ---> 2241
[290, 4920, 2241] ---> 287
[290, 4920, 2241, 287] ---> 257
 and --->  established
 and established --->  himself
 and established himself --->  in
 and established himself in --->  a


As a last step to enable embeddings, we need an efficient data loader that iterates over the input dataset and returns the inputs and targets as PyTorch tensors. We want to return two tensors:

- an input tensor containing the text that the LLM sees.
- an output, target sensor that includes the targets for the LLM to predict.

Each row in the (of size `max_length`) input tensor `x` represents one input context. A second tensor,`y` contains the corresponding prediction targets, which are created by shifting the input by one position.

In [28]:
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDataset(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []
        token_ids = tokenizer.encode(txt) #tokenizes entire text, and converts them into token IDs as a single step.
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk)) # use a sliding window to chunk the book into overlapping sequences of max_length

    def __len__(self):
        return len(self.input_ids) # returns the total number of row in the dataset

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx] #return a single row from dataset.

def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDataset(txt, tokenizer, max_length=max_length, stride=stride)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle,
                            drop_last=drop_last, #drop_last=True drops the last batch if it is shorter than the specified batch_size to prevent loss spikes during training.
                            num_workers=num_workers) # num of CPU processes to use for preprocessing.
    return dataloader

We test the `dataloader` with a batch size of 1 for an LLM with a context size of 4 to develop an intuition of how the `GPTDatasetV1` class and the `create_dataloader_v1` work together.

In [31]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)
data_iter = iter(dataloader) # convert dataloader into a iterator to fetch the next entry, leveraging the defined __getitem__ function above.
first_batch = next(data_iter)
print(first_batch[0]) #the input token IDs
print(first_batch[1]) #the target token IDs.

#Since max_length is set to 4, each of the two tensors contain four token IDs.

tensor([[  40,  367, 2885, 1464]])
tensor([[ 367, 2885, 1464]])


In [32]:
second_batch = next(data_iter)
print(second_batch[0])
print(second_batch[1])

tensor([[ 367, 2885, 1464, 1807]])
tensor([[2885, 1464, 1807]])


If we compare the first and second batches, we see that the second batch's token IDs are shifted by only one positions. The `stride` parameter dictates the number of positions the inputs shift across batches, emulating a sliding window approach.
